In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os


Using TensorFlow backend.


In [2]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('acc') > 0.98):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(0.98*100))   
            self.model.stop_training = True
callback= myCallback()

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    directory = "../data/train",
    target_size=(150, 150),
    batch_size=8,
    class_mode="categorical")


validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    directory = '../data/test/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 5503 images belonging to 4 classes.
Found 600 images belonging to 4 classes.


In [4]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(4, activation='sigmoid')
])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.compile(loss='mean_squared_error',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

Instructions for updating:
Use tf.cast instead.


In [6]:
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=100,
    callbacks=[callback]) # callbacks=[callback] (use it in the fit_generator)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
688/688 [==============================] - 186s 270ms/step - loss: 0.1256 - acc: 0.6260 - val_loss: 0.1261 - val_acc: 0.6200
Epoch 2/20
688/688 [==============================] - 193s 281ms/step - loss: 0.0858 - acc: 0.7583 - val_loss: 0.1195 - val_acc: 0.6733
Epoch 3/20
688/688 [==============================] - 199s 290ms/step - loss: 0.0786 - acc: 0.7770 - val_loss: 0.1351 - val_acc: 0.6500
Epoch 4/20
688/688 [==============================] - 184s 267ms/step - loss: 0.0745 - acc: 0.7878 - val_loss: 0.1280 - val_acc: 0.6667
Epoch 5/20
688/688 [==============================] - 158s 230ms/step - loss: 0.0703 - acc: 0.8007 - val_loss: 0.1309 - val_acc: 0.6550
Epoch 6/20
688/688 [==============================] - 158s 230ms/step - loss: 0.0669 - acc: 0.8181 - val_loss: 0.0956 - val_acc: 0.7467
Epoch 7/20
688/688 [==============================] - 159s 231ms/step - loss: 0.0640 - acc: 0.8274 - val_loss: 0.1251 - val_acc: 0.6783


In [7]:
evaluate=model.evaluate_generator(generator=validation_generator,steps=300)
evaluate

[0.14167327767139987, 0.68333334]

In [8]:
def find_class(result_list):
    if result_list[0] == 1 :
        return "Bacteria"
    elif result_list[1] == 1 : 
        return "Covid"
    elif result_list[2] == 1 : 
        return "Normal"
    elif result_list[3] == 1 : 
        return "Virus"
    else:
        return "Sınıf Tespit edilemedi"

In [10]:
basepath = '../data/predict/'
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        print(entry)
        img = image.load_img(os.path.join(basepath, entry), target_size=(150, 150))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        images = np.vstack([x])
        classes = model.predict(images, batch_size=10)
        print("Tahmin = " + find_class(classes[0]))
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXX")
        

COVID-19 (1).png
Tahmin = Virus
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (10).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (100).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (104).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (11).png
Tahmin = Virus
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (12).png
Tahmin = Bacteria
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (13).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (14).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (15).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (16).png
Tahmin = Bacteria
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (17).png
Tahmin = Virus
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (18).png
Tahmin = Covid
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (19).png
Tahmin = Sınıf Tespit edilemedi
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (2).png
Tahmin = Normal
XXXXXXXXXXXXXXXXXXXXXXXXXXX
COVID-19 (20).png
Ta